# Complete Medical LLM Assistant Pipeline

**Author:** Yassin Hagenimana  
**Project:** Fine-tuning TinyLlama for Medical Question-Answering  
**Dataset:** Medical Meadow Medical Flashcards (5,000 samples)  
**Method:** LoRA (Low-Rank Adaptation)  
**Platform:** Google Colab (T4 GPU)

---

## Overview

This comprehensive notebook contains the complete pipeline for building a medical question-answering LLM:

1. **Data Exploration** - Analyze the medical dataset
2. **Data Preprocessing** - Clean and format data for training
3. **Model Fine-tuning** - Train TinyLlama with LoRA
4. **Evaluation** - Compare base vs fine-tuned model
5. **Inference Demo** - Test the model with real questions

**Estimated Total Runtime:** 3-5 hours on Google Colab T4 GPU

---

## Table of Contents

- [Section 1: Data Exploration](#section-1)
- [Section 2: Data Preprocessing](#section-2)
- [Section 3: Model Fine-tuning](#section-3)
- [Section 4: Evaluation](#section-4)
- [Section 5: Inference Demo](#section-5)

---
# Section 1: Data Exploration <a id="section-1"></a>

In this section, we explore the **Medical Meadow Medical Flashcards** dataset to understand:
- Dataset size and structure
- Question and answer formats
- Data quality and distribution
- Sample medical content

This helps us make informed decisions about preprocessing and training.

## 1.1 Check GPU Availability

**What this does:** Verifies that a CUDA-compatible GPU is available for training.

**Why it's important:** Training language models requires GPU acceleration. Without a GPU, training would take days instead of hours.

**Expected output:** Should show `CUDA available: True` on Google Colab with GPU runtime.

In [ ]:
import os
import sys
from pathlib import Path
from datetime import datetime

# Add project root to Python path to import source modules
project_root = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
sys.path.insert(0, str(project_root))

# Import custom logger from source
from src.utils.logger import setup_logger, ExperimentLogger

# Create results folder structure
base_dir = Path('results')
folders = [
    base_dir / 'visualizations',
    base_dir / 'visualizations' / 'data_exploration',
    base_dir / 'visualizations' / 'preprocessing',
    base_dir / 'visualizations' / 'training',
    base_dir / 'visualizations' / 'evaluation',
    base_dir / 'visualizations' / 'inference',
    base_dir / 'models',
    base_dir / 'metrics',
    base_dir / 'experiments'
]

# Create all folders
for folder in folders:
    folder.mkdir(parents=True, exist_ok=True)
    print(f'✓ Created: {folder}')

# Setup experiment logger using source module
experiment_id = datetime.now().strftime('%Y%m%d_%H%M%S')
experiment_logger = ExperimentLogger(
    experiment_name=f'medical_llm_{experiment_id}',
    log_dir=str(base_dir / 'experiments')
)

print(f'\n Experiment ID: {experiment_id}')
print(f' Results will be saved to: {base_dir.absolute()}')
print(f' Logging initialized with ExperimentLogger')

## 1.0 Setup Results Folder Structure

**What this does:** Creates organized folders to save all visualizations, models, and results.

**Folder structure:**
- `results/` - Main results directory
- `results/visualizations/` - All plots and charts
- `results/models/` - Saved models
- `results/metrics/` - Evaluation metrics and tables
- `results/experiments/` - Experiment tracking logs

**Why it's important:** Organized file structure makes it easy to find and share results.

In [ ]:
# Import PyTorch to check GPU availability
import torch

# Check if CUDA (GPU) is available
print(f'CUDA available: {torch.cuda.is_available()}')

# If GPU is available, print GPU details
if torch.cuda.is_available():
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

## 1.2 Load the Medical Dataset

**What this does:** Downloads the Medical Meadow Medical Flashcards dataset from HuggingFace.

**Dataset details:**
- **Name:** medalpaca/medical_meadow_medical_flashcards
- **Total size:** 33,955 medical Q&A pairs
- **Format:** Each example has 'input' (question) and 'output' (answer)
- **Content:** Medical flashcards covering various medical topics

**Why this dataset:** High-quality medical content suitable for fine-tuning medical Q&A models.

In [ ]:
# Import DatasetLoader from source module
from src.data_processing.loader import DatasetLoader

# Initialize data loader with dataset name
print('Loading Medical Meadow Medical Flashcards dataset...')
data_loader = DatasetLoader(
    dataset_name='medalpaca/medical_meadow_medical_flashcards'
)

# Load the dataset using the custom loader
dataset = data_loader.load_dataset()

# Display total number of examples in the dataset
print(f'\nTotal examples in dataset: {len(dataset["train"]):,}')

# Show dataset structure
print(f'\nDataset structure: {dataset}')

# Log dataset info
experiment_logger.logger.info(f'Dataset loaded: {data_loader.dataset_name}')
experiment_logger.logger.info(f'Total examples: {len(dataset["train"]):,}')

## 1.3 Examine Sample Data

**What this does:** Displays a few sample questions and answers from the dataset.

**Why it's important:** Understanding the format and quality of data helps us:
- Verify data quality
- Design appropriate preprocessing steps
- Understand the complexity of medical content
- Determine suitable prompt formatting

In [ ]:
# Display 3 sample examples from the dataset
print('SAMPLE MEDICAL Q&A PAIRS')
print('=' * 80)

for i in range(3):
    example = dataset['train'][i]
    print(f'\nExample {i+1}:')
    print(f"Question: {example['input']}")
    print(f"Answer: {example['output']}")
    print('-' * 80)

## 1.4 Analyze Data Statistics

**What this does:** Calculates statistical properties of the dataset including:
- Average question length
- Average answer length
- Distribution of text lengths

**Why it's important:** This helps us choose appropriate:
- Maximum sequence length for tokenization
- Batch size for training
- Model configuration parameters

In [ ]:
import pandas as pd
import numpy as np

# Convert dataset to pandas DataFrame for easier analysis
df = pd.DataFrame(dataset['train'])

# Calculate text lengths
df['question_length'] = df['input'].str.len()
df['answer_length'] = df['output'].str.len()
df['total_length'] = df['question_length'] + df['answer_length']

# Display statistics
print('DATASET STATISTICS')
print('=' * 80)
print(f'Total examples: {len(df):,}')
print(f'\nQuestion Length:')
print(f'  Average: {df["question_length"].mean():.1f} characters')
print(f'  Median: {df["question_length"].median():.1f} characters')
print(f'  Max: {df["question_length"].max()} characters')
print(f'\nAnswer Length:')
print(f'  Average: {df["answer_length"].mean():.1f} characters')
print(f'  Median: {df["answer_length"].median():.1f} characters')
print(f'  Max: {df["answer_length"].max()} characters')
print(f'\nTotal Length (Q+A):')
print(f'  Average: {df["total_length"].mean():.1f} characters')
print(f'  95th percentile: {df["total_length"].quantile(0.95):.1f} characters')

## 1.5 Visualize Data Distribution

**What this does:** Creates visualizations showing the distribution of text lengths.

**Why it's important:** Visual analysis helps us:
- Identify outliers (very long or short texts)
- Choose appropriate max_length for tokenization
- Understand data balance

In [ ]:
import matplotlib.pyplot as plt

# Create visualization of text length distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot question length distribution
axes[0].hist(df['question_length'], bins=50, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Question Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Question Length Distribution')
axes[0].grid(alpha=0.3)

# Plot answer length distribution
axes[1].hist(df['answer_length'], bins=50, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('Answer Length (characters)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Answer Length Distribution')
axes[1].grid(alpha=0.3)

# Plot total length distribution
axes[2].hist(df['total_length'], bins=50, color='lightgreen', edgecolor='black')
axes[2].set_xlabel('Total Length (characters)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Total Q+A Length Distribution')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print('Distribution plots generated successfully!')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from collections import Counter
import re

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Sample 5000 examples for analysis
subset = dataset['train'].select(range(5000))

# Calculate text lengths
question_lengths = [len(ex['input'].split()) for ex in subset]
answer_lengths = [len(ex['output'].split()) for ex in subset]

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Medical Dataset - Comprehensive Data Analysis', fontsize=16, fontweight='bold')

# 1. Question Length Distribution
axes[0, 0].hist(question_lengths, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Number of Words', fontsize=12)
axes[0, 0].set_ylabel('Frequency', fontsize=12)
axes[0, 0].set_title('Question Length Distribution', fontsize=14, fontweight='bold')
axes[0, 0].axvline(np.mean(question_lengths), color='red', linestyle='--', 
                    label=f'Mean: {np.mean(question_lengths):.1f}')
axes[0, 0].legend()

# 2. Answer Length Distribution
axes[0, 1].hist(answer_lengths, bins=50, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Number of Words', fontsize=12)
axes[0, 1].set_ylabel('Frequency', fontsize=12)
axes[0, 1].set_title('Answer Length Distribution', fontsize=14, fontweight='bold')
axes[0, 1].axvline(np.mean(answer_lengths), color='red', linestyle='--', 
                    label=f'Mean: {np.mean(answer_lengths):.1f}')
axes[0, 1].legend()

# 3. Question-Answer Length Correlation
axes[1, 0].scatter(question_lengths, answer_lengths, alpha=0.3, s=10, color='green')
axes[1, 0].set_xlabel('Question Length (words)', fontsize=12)
axes[1, 0].set_ylabel('Answer Length (words)', fontsize=12)
axes[1, 0].set_title('Question vs Answer Length Correlation', fontsize=14, fontweight='bold')
# Add correlation coefficient
corr = np.corrcoef(question_lengths, answer_lengths)[0, 1]
axes[1, 0].text(0.05, 0.95, f'Correlation: {corr:.3f}', 
                transform=axes[1, 0].transAxes, fontsize=11,
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# 4. Top 20 Most Common Words (Medical Terms)
# Combine all text and extract words
all_text = ' '.join([ex['input'] + ' ' + ex['output'] for ex in subset])
# Remove common stopwords and extract medical terms
words = re.findall(r'\b[a-zA-Z]{4,}\b', all_text.lower())
word_freq = Counter(words).most_common(20)
words_list, counts = zip(*word_freq)

axes[1, 1].barh(range(len(words_list)), counts, color='purple', alpha=0.7)
axes[1, 1].set_yticks(range(len(words_list)))
axes[1, 1].set_yticklabels(words_list)
axes[1, 1].set_xlabel('Frequency', fontsize=12)
axes[1, 1].set_title('Top 20 Most Common Words', fontsize=14, fontweight='bold')
axes[1, 1].invert_yaxis()

plt.tight_layout()

# Save the figure
save_path = 'results/visualizations/data_exploration/comprehensive_data_analysis.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f'✓ Saved: {save_path}')
plt.show()

# Create and save summary statistics table
stats_df = pd.DataFrame({
    'Metric': [
        'Total Samples',
        'Avg Question Length (words)',
        'Avg Answer Length (words)',
        'Max Question Length',
        'Max Answer Length',
        'Min Question Length',
        'Min Answer Length',
        'Median Question Length',
        'Median Answer Length',
        'Std Dev Question Length',
        'Std Dev Answer Length'
    ],
    'Value': [
        len(subset),
        f'{np.mean(question_lengths):.2f}',
        f'{np.mean(answer_lengths):.2f}',
        max(question_lengths),
        max(answer_lengths),
        min(question_lengths),
        min(answer_lengths),
        np.median(question_lengths),
        np.median(answer_lengths),
        f'{np.std(question_lengths):.2f}',
        f'{np.std(answer_lengths):.2f}'
    ]
})

# Save statistics table
stats_path = 'results/metrics/data_exploration_statistics.csv'
stats_df.to_csv(stats_path, index=False)
print(f'✓ Saved: {stats_path}')

# Display the table
print('\n' + '='*50)
print('DATA EXPLORATION STATISTICS')
print('='*50)
print(stats_df.to_string(index=False))
print('='*50)

## 1.6 Comprehensive Data Visualizations

**What this does:** Creates detailed visualizations of the dataset to understand:
- Text length distributions (questions and answers)
- Word frequency analysis
- Data quality metrics
- Sample distributions

**Visualizations saved:**
- Question length histogram
- Answer length histogram
- Top 20 most common words (bar chart)
- Question-Answer length correlation scatter plot
- Summary statistics table

**Why it's important:** Visual analysis helps identify data patterns, outliers, and quality issues.

---
# Section 2: Data Preprocessing <a id="section-2"></a>

In this section, we prepare the data for training:
- Load and clean the dataset
- Select a subset (5,000 examples) for efficient training
- Format data in instruction-response format
- Split into train/validation/test sets
- Save processed data

**Goal:** Create high-quality training data for fine-tuning.

## 2.1 Import Required Libraries

**What this does:** Imports all necessary libraries for data preprocessing.

**Libraries used:**
- `pandas`: Data manipulation and analysis
- `datasets`: HuggingFace datasets library
- `transformers`: For tokenizer (optional check)
- `warnings`: Suppress warning messages for cleaner output

In [ ]:
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import warnings
import os

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

print('Libraries imported successfully!')

## 2.2 Load and Inspect Dataset

**What this does:** Loads the dataset and converts it to a pandas DataFrame for easier manipulation.

**Why DataFrame:** Pandas provides powerful tools for:
- Data cleaning
- Duplicate removal
- Filtering
- Statistical analysis

In [ ]:
# Load the medical flashcards dataset
print('Loading dataset...')
dataset = load_dataset('medalpaca/medical_meadow_medical_flashcards')

# Convert to pandas DataFrame
df = pd.DataFrame(dataset['train'])

print(f'Total examples loaded: {len(df):,}')
print(f'Columns: {list(df.columns)}')
print(f'\nFirst few rows:')
print(df.head())

## 2.3 Clean the Data

**What this does:** Removes duplicates and empty entries to ensure data quality.

**Cleaning steps:**
1. Remove duplicate question-answer pairs
2. Remove entries with empty questions
3. Remove entries with empty answers

**Why it's important:** Clean data leads to better model performance and prevents overfitting on duplicates.

In [ ]:
# Import DataPreprocessor from source module
from src.data_processing.preprocessor import DataPreprocessor

# Initialize preprocessor
preprocessor = DataPreprocessor(
    instruction_template="### Instruction:\n{instruction}\n\n### Response:\n{response}",
    max_length=512
)

print(f'Original size: {len(df):,} examples')

# Remove duplicate rows
df_clean = df.drop_duplicates()
print(f'After removing duplicates: {len(df_clean):,} examples')

# Remove rows with empty questions or answers AND clean text using preprocessor
df_clean['input'] = df_clean['input'].apply(preprocessor.clean_text)
df_clean['output'] = df_clean['output'].apply(preprocessor.clean_text)

# Remove empty entries after cleaning
df_clean = df_clean[
    (df_clean['input'].str.len() > 0) & 
    (df_clean['output'].str.len() > 0)
]

print(f'After cleaning and removing empty entries: {len(df_clean):,} examples')
print(f'\nRemoved {len(df) - len(df_clean):,} low-quality examples')

# Log preprocessing step
experiment_logger.logger.info(f'Data cleaned: {len(df_clean):,} examples remaining')

## 2.4 Select Subset for Efficient Training

**What this does:** Randomly selects 5,000 examples from the cleaned dataset.

**Why 5,000 examples:**
- Balances training quality with time constraints
- Fits comfortably in Google Colab's free tier
- Sufficient for effective fine-tuning with LoRA
- Allows completion within 2-4 hours

**Random seed (42):** Ensures reproducibility - same subset every time.

In [ ]:
# Define subset size
SUBSET_SIZE = 1000

# Randomly sample 10000 examples (with fixed random seed for reproducibility)
df_subset = df_clean.sample(n=SUBSET_SIZE, random_state=42)

print(f'Selected subset size: {len(df_subset):,} examples')
print(f'Percentage of full dataset: {(len(df_subset) / len(df_clean) * 100):.1f}%')

## 2.5 Format as Instruction-Response

**What this does:** Converts raw Q&A pairs into instruction-following format.

**Format used:**
```
### Instruction:
[Question]

### Response:
[Answer]
```

**Why this format:**
- Explicitly separates instruction from response
- Compatible with instruction-tuned models like TinyLlama
- Helps model understand its role
- Standard format for fine-tuning chat/instruction models

In [ ]:
# Use DataPreprocessor to format instruction-response pairs
def format_instruction(row):
    """Convert Q&A pair to instruction-response format using preprocessor"""
    return preprocessor.format_instruction_response(
        instruction=row['input'],
        response=row['output']
    )

# Apply formatting to all rows
df_subset['text'] = df_subset.apply(format_instruction, axis=1)

# Display a sample formatted example
print('SAMPLE FORMATTED EXAMPLE:')
print('=' * 80)
print(df_subset['text'].iloc[0])
print('=' * 80)
print(f'\nTotal formatted examples: {len(df_subset):,}')

# Log formatting step
experiment_logger.logger.info(f'Data formatted: {len(df_subset):,} examples')

## 2.6 Split into Train/Validation/Test Sets

**What this does:** Divides data into three sets for training, validation, and testing.

**Split ratios:**
- **Train (85%):** 4,250 examples - Used to train the model
- **Validation (10%):** 500 examples - Monitor training progress, prevent overfitting
- **Test (5%):** 250 examples - Final evaluation of model performance

**Why this split:**
- Train: Largest portion for learning patterns
- Validation: Detect overfitting during training
- Test: Unbiased evaluation of final model

In [ ]:
# First split: 85% train, 15% temporary (for val+test)
train_df, temp_df = train_test_split(
    df_subset, 
    test_size=0.15, 
    random_state=42
)

# Second split: Split the 15% into validation (10%) and test (5%)
# 0.33 of 15% ≈ 5% of total
val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.33, 
    random_state=42
)

# Display split statistics
print('DATASET SPLIT COMPLETED')
print('=' * 80)
print(f'Train set:      {len(train_df):,} examples ({len(train_df)/len(df_subset)*100:.1f}%)')
print(f'Validation set: {len(val_df):,} examples ({len(val_df)/len(df_subset)*100:.1f}%)')
print(f'Test set:       {len(test_df):,} examples ({len(test_df)/len(df_subset)*100:.1f}%)')
print(f'Total:          {len(train_df) + len(val_df) + len(test_df):,} examples')

## 2.7 Save Processed Data

**What this does:** Saves the processed datasets to CSV files.

**Files created:**
- `../data/processed/train.csv`: Training data
- `../data/processed/val.csv`: Validation data
- `../data/processed/test.csv`: Test data

**Why save to CSV:**
- Can be loaded quickly without re-processing
- Easy to inspect and share
- Compatible with various tools
- Serves as backup of processed data

In [ ]:
# Create directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

# Save each split to CSV (only input and output columns)
train_df[['input', 'output']].to_csv('../data/processed/train.csv', index=False)
val_df[['input', 'output']].to_csv('../data/processed/val.csv', index=False)
test_df[['input', 'output']].to_csv('../data/processed/test.csv', index=False)


In [ ]:
# Convert pandas DataFrames to HuggingFace Dataset objects
train_dataset = Dataset.from_pandas(train_df[['text']])
val_dataset = Dataset.from_pandas(val_df[['text']])
test_dataset = Dataset.from_pandas(test_df[['text']])

print('DATASETS CONVERTED SUCCESSFULLY!')
print('=' * 80)
print(f'Train dataset:      {len(train_dataset):,} examples')
print(f'Validation dataset: {len(val_dataset):,} examples')
print(f'Test dataset:       {len(test_dataset):,} examples')
print('=' * 80)
print('\nDataset objects ready for training!')

# Log dataset conversion
experiment_logger.logger.info(f'Datasets converted: train={len(train_dataset)}, val={len(val_dataset)}, test={len(test_dataset)}')

## 2.8 Convert to HuggingFace Dataset Format

**What this does:** Converts pandas DataFrames to HuggingFace Dataset objects.

**Why this is needed:**
- HuggingFace Datasets are optimized for ML workflows
- Required format for tokenization and training
- Enables efficient data loading and batching
- Compatible with Transformers library

## 2.9 Visualize Data Splits

**What this does:** Creates comprehensive visualizations of the train/validation/test split.

**Visualizations created:**
- **Pie chart:** Shows proportions of each split
- **Bar chart:** Compares sample counts across splits

**Outputs:**
- `results/visualizations/preprocessing/data_split_analysis.png` - Split visualization
- `results/metrics/preprocessing_summary.csv` - Summary statistics table

**Why visualize:** Helps verify correct data splitting and provides documentation for reports.

In [ ]:
# Create preprocessing visualizations
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle('Data Preprocessing - Train/Val/Test Split Analysis', fontsize=16, fontweight='bold')

# Get split sizes
train_size = len(train_dataset)
val_size = len(val_dataset)
test_size = len(test_dataset)
total_size = train_size + val_size + test_size

# 1. Pie Chart - Split Proportions
sizes = [train_size, val_size, test_size]
labels = [f'Train\n({train_size:,} samples)\n{train_size/total_size*100:.1f}%',
          f'Validation\n({val_size:,} samples)\n{val_size/total_size*100:.1f}%',
          f'Test\n({test_size:,} samples)\n{test_size/total_size*100:.1f}%']
colors = ['#3498db', '#2ecc71', '#e74c3c']
explode = (0.05, 0.05, 0.05)

axes[0].pie(sizes, explode=explode, labels=labels, colors=colors,
            autopct='', startangle=90, textprops={'fontsize': 11, 'weight': 'bold'})
axes[0].set_title('Dataset Split Proportions', fontsize=14, fontweight='bold')

# 2. Bar Chart - Split Comparison
split_names = ['Train', 'Validation', 'Test']
bars = axes[1].bar(split_names, sizes, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Number of Samples', fontsize=12, fontweight='bold')
axes[1].set_title('Dataset Split Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylim(0, max(sizes) * 1.1)

# Add value labels on bars
for i, (bar, size) in enumerate(zip(bars, sizes)):
    height = bar.get_height()
    axes[1].text(bar.get_x() + bar.get_width()/2., height,
                f'{size:,}\n({size/total_size*100:.1f}%)',
                ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add grid for better readability
axes[1].grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()

# Save the figure
save_path = 'results/visualizations/preprocessing/data_split_analysis.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f'✓ Saved: {save_path}')
plt.show()

# Create preprocessing summary table
prep_summary_df = pd.DataFrame({
    'Split': ['Training', 'Validation', 'Test', 'Total'],
    'Samples': [train_size, val_size, test_size, total_size],
    'Percentage': [f'{train_size/total_size*100:.1f}%', 
                   f'{val_size/total_size*100:.1f}%',
                   f'{test_size/total_size*100:.1f}%',
                   '100.0%'],
    'Purpose': [
        'Model training and weight updates',
        'Hyperparameter tuning and early stopping',
        'Final unbiased model evaluation',
        'Complete dataset'
    ]
})

# Save preprocessing summary
prep_path = 'results/metrics/preprocessing_summary.csv'
prep_summary_df.to_csv(prep_path, index=False)
print(f'✓ Saved: {prep_path}')

# Display the table
print('\n' + '='*80)
print('PREPROCESSING SUMMARY')
print('='*80)
print(prep_summary_df.to_string(index=False))
print('='*80)

# Display sample formatted data
print('\n' + '='*80)
print('SAMPLE FORMATTED DATA (First Example)')
print('='*80)
print(train_dataset[0]['text'][:500] + '...')
print('='*80)

## 2.7 Preprocessing Visualizations

**What this does:** Creates visualizations showing:
- Train/Validation/Test split proportions (pie chart)
- Data distribution across splits (bar chart)
- Sample formatted data display

**Visualizations saved:**
- Data split pie chart
- Split comparison bar chart
- Preprocessing summary table

**Why it's important:** Confirms proper data splitting and formatting for training.

---
# Section 3: Model Fine-tuning with LoRA <a id="section-3"></a>

In this section, we fine-tune TinyLlama-1.1B using LoRA (Low-Rank Adaptation).

**What is LoRA:**
- Efficient fine-tuning method that trains only a small subset of parameters
- Reduces memory usage by ~70%
- Maintains performance quality
- Perfect for training on limited GPU resources (Google Colab)

**Training time:** Approximately 2-4 hours on Colab T4 GPU

**Key configurations:**
- 8-bit quantization for memory efficiency
- LoRA rank=16, alpha=32
- 3 epochs of training
- Batch size=2 with gradient accumulation=4

## 3.1 Check GPU Availability

**What this does:** Verifies GPU is available and displays GPU specifications.

**Important:** This step is critical - without GPU, training will fail or take days.

In [ ]:
import torch

print('GPU AVAILABILITY CHECK')
print('=' * 80)
print(f'CUDA available: {torch.cuda.is_available()}')

if torch.cuda.is_available():
    print(f'GPU Name: {torch.cuda.get_device_name(0)}')
    print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')
    print('\nGPU is ready for training!')
else:
    print('\nWARNING: No GPU detected!')
    print('Please enable GPU in Colab: Runtime -> Change runtime type -> GPU')

## 3.2 Install Required Packages

**What this does:** Installs the latest versions of essential libraries.

**Libraries installed:**
- `transformers`: HuggingFace library for LLMs
- `peft`: Parameter-Efficient Fine-Tuning (includes LoRA)
- `datasets`: For dataset handling
- `accelerate`: For distributed training
- `bitsandbytes`: For 8-bit quantization
- `trl`: Transformer Reinforcement Learning (improved training)

**Note:** The `-q` flag makes installation quiet (less verbose output).

In [ ]:
# Install required packages
!pip install -q transformers>=4.35.0 peft>=0.7.0 datasets>=2.14.0 accelerate>=0.24.0 bitsandbytes>=0.41.0 trl>=0.7.0

print('All required packages installed successfully!')

## 3.3 Use Preprocessed Data

**What this does:** Reuses the datasets already created in Section 2.

**Advantages:**
- No duplicate data loading
- Consistent datasets across sections
- Saves time and memory
- Already cleaned, formatted, and split

**Datasets available:**
- `train_dataset`: 4,250 examples (85%)
- `val_dataset`: 500 examples (10%)
- `test_dataset`: 250 examples (5%)

**Note:** These datasets are already formatted in instruction-response format and ready for tokenization.

In [ ]:
# Use the datasets already prepared in Section 2
# No need to reload and reprocess the data!

print('Using datasets from Section 2...')
print(f'Training set:   {len(train_dataset):,} examples')
print(f'Validation set: {len(val_dataset):,} examples') 
print(f'Test set:       {len(test_dataset):,} examples')

print(f'\nSample example:')
print(train_dataset[0]['text'][:300] + '...')

# Create aliases for consistency with rest of Section 3
train_data = train_dataset
val_data = val_dataset
test_data = test_dataset

print('\nDatasets ready for model training!')

## 3.6 Load Model and Tokenizer with 8-bit Quantization

**What this does:** Loads TinyLlama model with memory-efficient 8-bit quantization.

**Model:** TinyLlama/TinyLlama-1.1B-Chat-v1.0
- Size: 1.1 billion parameters
- Type: Chat-optimized causal language model
- Why chosen: Fits in Colab's 15GB GPU memory with quantization

**8-bit Quantization benefits:**
- Reduces memory usage by ~50%
- Minimal performance loss (~1%)
- Enables training on limited GPU

**Configuration:**
- `load_in_8bit=True`: Use 8-bit precision
- `bnb_8bit_use_double_quant=True`: Further memory optimization
- `device_map='auto'`: Automatic GPU placement

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

MODEL_NAME = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

# Check if model is already loaded
if 'model' not in globals() or model is None:
    # Configure 8-bit quantization for memory efficiency
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True,                    # Use 8-bit precision
        bnb_8bit_use_double_quant=True,       # Double quantization for more memory savings
        bnb_8bit_quant_type='nf8',            # NormalFloat 8-bit quantization
        bnb_8bit_compute_dtype=torch.float16  # Compute in float16 for speed
    )

    print('Loading model with 8-bit quantization...')

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map='auto',           # Automatically place on GPU
        trust_remote_code=True       # Allow custom model code
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

    # Set padding token (required for batch training)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print('\nMODEL LOADED SUCCESSFULLY')
    print('=' * 80)
    print(f'Model: {MODEL_NAME}')
    print(f'Total parameters: {model.num_parameters():,}')
    print(f'Memory format: 8-bit quantized')
    print(f'Device: {next(model.parameters()).device}')
else:
    print('MODEL ALREADY LOADED')
    print('=' * 80)
    print(f'Model: {MODEL_NAME}')
    print(f'Total parameters: {model.num_parameters():,}')
    print(f'Memory format: 8-bit quantized')
    print(f'Device: {next(model.parameters()).device}')
    print('\nSkipping model loading to save time!')

## 3.7 Configure LoRA (Low-Rank Adaptation)

**What this does:** Sets up LoRA for efficient fine-tuning.

**What is LoRA:**
- Adds small trainable matrices (adapters) to model layers
- Freezes original model weights
- Trains only ~0.5% of total parameters
- Results in much faster training and lower memory usage

**LoRA Configuration:**
- `r=16`: Rank of low-rank matrices (higher = more capacity, more memory)
- `lora_alpha=32`: Scaling factor (typically 2x rank)
- `lora_dropout=0.05`: Dropout for regularization
- `target_modules`: Which attention layers to adapt
  - `q_proj`: Query projection
  - `k_proj`: Key projection
  - `v_proj`: Value projection
  - `o_proj`: Output projection

**Why these modules:** Attention layers contain most of the model's knowledge and adapt well to new tasks.

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

# Prepare model for k-bit (8-bit) training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
peft_config = LoraConfig(
    r=16,                                           # Low-rank dimension
    lora_alpha=32,                                  # Scaling factor (2 * r)
    lora_dropout=0.05,                              # Dropout probability
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj'],  # Attention layers to adapt
    bias='none',                                    # Don't adapt bias terms
    task_type='CAUSAL_LM'                          # Task: Causal Language Modeling
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)

# Display trainable parameters
print('LORA CONFIGURATION APPLIED')
print('=' * 80)
model.print_trainable_parameters()
print('\nOnly these LoRA adapters will be trained!')
print('Original model weights remain frozen.')

## 3.8 Tokenize Dataset

**What this does:** Converts text into token IDs that the model can process.

**Tokenization process:**
1. Convert text to token IDs using tokenizer
2. Truncate sequences longer than 512 tokens
3. Pad shorter sequences to 512 tokens
4. Create labels (copy of input_ids for causal LM)

**Parameters:**
- `max_length=512`: Maximum sequence length
  - Chosen based on dataset statistics (95th percentile)
  - Balances context vs memory usage
- `padding='max_length'`: Pad all sequences to same length for efficient batching
- `truncation=True`: Cut off sequences exceeding max_length

**Batched processing:** Processes multiple examples at once for speed.

In [ ]:
def tokenize_function(examples):
    """
    Tokenize text examples for training.
    
    Args:
        examples: Batch of examples with 'text' field
    
    Returns:
        Dictionary with input_ids, attention_mask, and labels
    """
    # Tokenize text
    outputs = tokenizer(
        examples['text'],
        truncation=True,          # Truncate long sequences
        max_length=512,           # Maximum sequence length
        padding='max_length',     # Pad to max_length
        return_tensors=None       # Return Python lists
    )
    
    # For causal LM, labels are the same as input_ids
    outputs['labels'] = outputs['input_ids'].copy()
    
    return outputs

print('Tokenizing datasets...')

# Tokenize training data
train_tokenized = train_data.map(
    tokenize_function, 
    batched=True,                          # Process in batches for speed
    remove_columns=train_data.column_names  # Remove all original columns after tokenization
)

# Tokenize validation data
val_tokenized = val_data.map(
    tokenize_function, 
    batched=True, 
    remove_columns=val_data.column_names
)

print('\nTOKENIZATION COMPLETE')
print('=' * 80)
print(f'Tokenized training examples:   {len(train_tokenized):,}')
print(f'Tokenized validation examples: {len(val_tokenized):,}')
print(f'Sequence length: {train_tokenized[0]["input_ids"].__len__()} tokens')
print('\nDatasets ready for training!')

## 3.9 Configure Training Parameters

**What this does:** Sets all hyperparameters for the training process.

**Key parameters explained:**

**Training length:**
- `num_train_epochs=1`: Train for 1 complete pass through data
  - Faster training for development and testing
  - Can increase to 2-3 epochs for final production model

**Batch sizes:**
- `per_device_train_batch_size=2`: Process 2 examples at once
- `gradient_accumulation_steps=4`: Accumulate gradients over 4 batches
- **Effective batch size = 2 × 4 = 8**
  - Small batch size fits in limited GPU memory
  - Gradient accumulation simulates larger batch

**Optimization:**
- `learning_rate=2e-4`: Step size for weight updates
  - Higher than typical (1e-5) because we're using LoRA
  - LoRA allows higher learning rates
- `lr_scheduler_type='cosine'`: Gradually decrease learning rate
- `warmup_steps=100`: Slowly increase LR at start

**Precision:**
- `fp16=True`: Use 16-bit floating point
  - 2x faster training
  - 2x less memory
  - Negligible accuracy loss

**Checkpointing:**
- `save_steps=500`: Save model every 500 steps
- `eval_steps=500`: Evaluate on validation set every 500 steps
- `load_best_model_at_end=True`: Keep best checkpoint

In [ ]:
from transformers import TrainingArguments, Trainer

# Define training configuration
training_args = TrainingArguments(
    # Output
    output_dir='./medical_llm_checkpoints',   # Where to save checkpoints
    
    # Training duration
    num_train_epochs=1,                       # Number of training epochs
    
    # Batch sizes
    per_device_train_batch_size=2,            # Batch size per GPU for training
    per_device_eval_batch_size=2,             # Batch size per GPU for evaluation
    gradient_accumulation_steps=4,            # Accumulate gradients over 4 batches
                                              # Effective batch size = 2 * 4 = 8
    
    # Learning rate
    learning_rate=2e-4,                       # Learning rate (higher for LoRA)
    lr_scheduler_type='cosine',               # Cosine learning rate schedule
    warmup_steps=100,                         # Warmup steps for learning rate
    
    # Precision
    fp16=True,                                # Use mixed precision training
    
    # Checkpointing and evaluation
    save_steps=500,                           # Save checkpoint every 500 steps
    eval_steps=500,                           # Evaluate every 500 steps
    logging_steps=50,                         # Log metrics every 50 steps
    logging_first_step=True,                  # Log from the very first step
    eval_strategy='steps',                    # Evaluate at regular intervals
    save_strategy='steps',                    # Save at regular intervals
    load_best_model_at_end=True,             # Load best checkpoint at end
    
    # Logging
    report_to='none',                         # Don't use external logging (wandb, etc.)
    disable_tqdm=False,                       # Keep progress bars enabled
    logging_strategy='steps'                  # Log at regular step intervals
)

print('TRAINING CONFIGURATION SET')
print('=' * 80)
print(f'Epochs: {training_args.num_train_epochs}')
print(f'Batch size per device: {training_args.per_device_train_batch_size}')
print(f'Gradient accumulation: {training_args.gradient_accumulation_steps}')
print(f'Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}')
print(f'Learning rate: {training_args.learning_rate}')
print(f'Mixed precision: {training_args.fp16}')
print(f'\nEstimated training time: 2-4 hours on Colab T4 GPU')

## 3.10 Train the Model

**What this does:** Runs the actual training process.

**Training process:**
1. Initialize Trainer with model, data, and configuration
2. Start training loop:
   - Forward pass: Generate predictions
   - Calculate loss: Compare with actual answers
   - Backward pass: Compute gradients
   - Update LoRA weights
3. Periodically evaluate on validation set
4. Save checkpoints

**What you'll see:**
- Progress bar showing steps completed
- Training loss (should decrease over time)
- Validation loss (should also decrease)
- Time per batch

**Expected behavior:**
- Training loss should steadily decrease
- Validation loss should decrease then plateau
- If validation loss increases, model may be overfitting

**Duration:** Approximately 2-4 hours for 3 epochs

In [ ]:
# Create a custom callback to display epoch progress
from transformers import TrainerCallback

class EpochProgressCallback(TrainerCallback):
    def on_epoch_begin(self, args, state, control, **kwargs):
        print(f"\n{'='*80}")
        print(f"EPOCH {int(state.epoch) + 1}/{int(args.num_train_epochs)} STARTING")
        print(f"{'='*80}")
    
    def on_epoch_end(self, args, state, control, **kwargs):
        print(f"\n{'='*80}")
        print(f"EPOCH {int(state.epoch)}/{int(args.num_train_epochs)} COMPLETED")
        print(f"{'='*80}\n")

# Initialize Trainer with callback
trainer = Trainer(
    model=model,                    # LoRA-enhanced model
    args=training_args,             # Training configuration
    train_dataset=train_tokenized,  # Tokenized training data
    eval_dataset=val_tokenized,     # Tokenized validation data
    callbacks=[EpochProgressCallback()]  # Add epoch progress callback
)

print('STARTING TRAINING')
print('=' * 80)
print('This will take approximately 2-4 hours.')
print('You can monitor progress below:')
print('  - Progress bars show step-by-step progress')
print('  - Epoch transitions will be clearly marked')
print('  - Loss metrics logged every 50 steps')
print('  - Evaluation runs every 500 steps')
print('Loss should decrease over time.')
print('=' * 80)
print()

# Start training
trainer.train()

print('\n' + '=' * 80)
print('TRAINING COMPLETE!')
print('=' * 80)

In [ ]:
# Extract training metrics from trainer state
import json

# Get training history
log_history = trainer.state.log_history

# Extract loss values
train_losses = [log['loss'] for log in log_history if 'loss' in log]
train_steps = [log['step'] for log in log_history if 'loss' in log]

# Create comprehensive training visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Training Progress - Comprehensive Analysis', fontsize=16, fontweight='bold')

# 1. Training Loss Curve
axes[0, 0].plot(train_steps, train_losses, linewidth=2, color='blue', marker='o', markersize=4)
axes[0, 0].set_xlabel('Training Steps', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Training Loss', fontsize=12, fontweight='bold')
axes[0, 0].set_title('Training Loss Over Time', fontsize=14, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, linestyle='--')
# Add annotations for first and last loss
if train_losses:
    axes[0, 0].annotate(f'Start: {train_losses[0]:.4f}', 
                        xy=(train_steps[0], train_losses[0]),
                        xytext=(10, 10), textcoords='offset points',
                        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.5),
                        fontsize=10)
    axes[0, 0].annotate(f'End: {train_losses[-1]:.4f}', 
                        xy=(train_steps[-1], train_losses[-1]),
                        xytext=(-60, 10), textcoords='offset points',
                        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5),
                        fontsize=10)

# 2. Learning Rate Schedule
learning_rates = [log['learning_rate'] for log in log_history if 'learning_rate' in log]
lr_steps = [log['step'] for log in log_history if 'learning_rate' in log]
if learning_rates:
    axes[0, 1].plot(lr_steps, learning_rates, linewidth=2, color='red', marker='s', markersize=4)
    axes[0, 1].set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Learning Rate', fontsize=12, fontweight='bold')
    axes[0, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[0, 1].grid(True, alpha=0.3, linestyle='--')
    axes[0, 1].ticklabel_format(style='scientific', axis='y', scilimits=(0,0))

# 3. Loss Reduction Percentage
if len(train_losses) > 1:
    loss_reduction = [(train_losses[0] - loss) / train_losses[0] * 100 for loss in train_losses]
    axes[1, 0].plot(train_steps, loss_reduction, linewidth=2, color='green', marker='^', markersize=4)
    axes[1, 0].set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Loss Reduction (%)', fontsize=12, fontweight='bold')
    axes[1, 0].set_title('Cumulative Loss Reduction', fontsize=14, fontweight='bold')
    axes[1, 0].grid(True, alpha=0.3, linestyle='--')
    axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)

# 4. Training Statistics Summary (Text Box)
axes[1, 1].axis('off')
if train_losses:
    initial_loss = train_losses[0]
    final_loss = train_losses[-1]
    avg_loss = np.mean(train_losses)
    min_loss = min(train_losses)
    loss_improvement = ((initial_loss - final_loss) / initial_loss) * 100
    
    stats_text = f'''
    TRAINING STATISTICS
    {'='*40}
    
    Initial Loss:     {initial_loss:.6f}
    Final Loss:       {final_loss:.6f}
    Average Loss:     {avg_loss:.6f}
    Minimum Loss:     {min_loss:.6f}
    
    Loss Improvement: {loss_improvement:.2f}%
    Total Steps:      {train_steps[-1]:,}
    Total Epochs:     {training_args.num_train_epochs}
    
    Model:            TinyLlama-1.1B-Chat
    Method:           LoRA Fine-tuning
    Samples:          {len(train_dataset):,}
    
    LoRA Config:
      - Rank (r):     {peft_config.r}
      - Alpha:        {peft_config.lora_alpha}
      - Dropout:      {peft_config.lora_dropout}
    
    Training Args:
      - Batch Size:   {training_args.per_device_train_batch_size}
      - Learning Rate: {training_args.learning_rate}
      - Gradient Acc:  {training_args.gradient_accumulation_steps}
    '''
    
    axes[1, 1].text(0.1, 0.5, stats_text, fontsize=11, family='monospace',
                    verticalalignment='center',
                    bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.3))

plt.tight_layout()

# Save training visualization
save_path = 'results/visualizations/training/training_progress.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f'✓ Saved: {save_path}')
plt.show()

# Create and save training metrics table
if train_losses:
    training_metrics_df = pd.DataFrame({
        'Metric': [
            'Initial Loss',
            'Final Loss',
            'Average Loss',
            'Minimum Loss',
            'Loss Improvement (%)',
            'Total Steps',
            'Total Epochs',
            'Training Samples',
            'Batch Size',
            'Learning Rate',
            'Gradient Accumulation',
            'LoRA Rank',
            'LoRA Alpha',
            'LoRA Dropout'
        ],
        'Value': [
            f'{initial_loss:.6f}',
            f'{final_loss:.6f}',
            f'{avg_loss:.6f}',
            f'{min_loss:.6f}',
            f'{loss_improvement:.2f}%',
            train_steps[-1],
            training_args.num_train_epochs,
            len(train_dataset),
            training_args.per_device_train_batch_size,
            training_args.learning_rate,
            training_args.gradient_accumulation_steps,
            peft_config.r,
            peft_config.lora_alpha,
            peft_config.lora_dropout
        ]
    })
    
    # Save metrics
    metrics_path = 'results/metrics/training_metrics.csv'
    training_metrics_df.to_csv(metrics_path, index=False)
    print(f' Saved: {metrics_path}')
    
    # Display table
    print('\n' + '='*60)
    print('TRAINING METRICS SUMMARY')
    print('='*60)
    print(training_metrics_df.to_string(index=False))
    print('='*60)

## 3.14 Training Metrics Tracking and Visualization

**What this does:** Creates comprehensive visualizations of the training process:
- Training loss curve over time
- Learning rate schedule
- Training progress metrics
- GPU memory usage over time (if available)

**Visualizations saved:**
- Training loss curve
- Learning rate schedule
- Training metrics table
- Experiment configuration table

**Why it's important:** Visualizing training helps identify:
- If the model is learning (loss decreasing)
- If there's overfitting
- If hyperparameters are appropriate

## 3.11 Save the Fine-tuned Model

**What this does:** Saves the trained LoRA adapters and tokenizer to disk.

**What gets saved:**
- LoRA adapter weights (~100-200 MB)
- Adapter configuration
- Tokenizer files

**What doesn't get saved:**
- Base model (TinyLlama) - will be loaded separately
- Only the LoRA adapters are saved (much smaller)

**Location:** `./medical_llm_final/`

**Important:** Download this folder before Colab session ends!

In [ ]:
# Save LoRA adapters
save_directory = './medical_llm_final'

print('Saving model...')
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

print('\nMODEL SAVED SUCCESSFULLY')
print('=' * 80)
print(f'Location: {save_directory}/')
print('\nFiles saved:')
print('  - adapter_config.json   (LoRA configuration)')
print('  - adapter_model.bin     (LoRA weights)')
print('  - tokenizer files')
print('\nIMPORTANT: Download this folder before ending Colab session!')
print('You can also upload to Google Drive or HuggingFace Hub.')

## 3.12 Test the Fine-tuned Model

**What this does:** Generates a test response to verify the model works.

**Test question:** "What are the symptoms of diabetes?"

**What to expect:**
- Model should generate a medically accurate response
- Response should be relevant to diabetes symptoms
- Quality may not be perfect but should show medical knowledge

**Parameters:**
- `max_new_tokens=100`: Generate up to 100 new tokens
- `temperature=0.7`: Balance between creativity and focus
  - Lower (0.1) = more focused, deterministic
  - Higher (1.0) = more creative, diverse

In [ ]:
# Detect available device (GPU or CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Test the model with a sample question
test_question = 'What are the symptoms of diabetes?'

# Format in instruction format
input_text = f"""### Instruction:
{test_question}

### Response:
"""

# Tokenize input
inputs = tokenizer(input_text, return_tensors='pt').to(device)

# Generate response
print('TEST GENERATION')
print('=' * 80)
print(f'Question: {test_question}')
print('\nGenerating response...')

with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_new_tokens=100,     # Generate up to 100 tokens
        temperature=0.7,        # Sampling temperature
        do_sample=True,         # Use sampling instead of greedy
        top_p=0.9               # Nucleus sampling
    )

# Decode response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print('\nGenerated Response:')
print('-' * 80)
print(response)
print('=' * 80)
print('\nModel is working! Proceed to evaluation for detailed metrics.')

## 3.13 Training Complete - Next Steps

**Congratulations! Model fine-tuning is complete.**

**What you've accomplished:**
- ✓ Fine-tuned TinyLlama on 5,000 medical Q&A pairs
- ✓ Used LoRA for efficient training
- ✓ Saved model checkpoints
- ✓ Verified model generates responses

**Next steps:**

1. **Download Model:**
   - Right-click on `medical_llm_final` folder
   - Select "Download"
   - Or upload to Google Drive/HuggingFace Hub

2. **Run Evaluation (Section 4):**
   - Calculate BLEU and ROUGE scores
   - Compare with base model
   - Generate metrics report

3. **Test Inference (Section 5):**
   - Try various medical questions
   - Experiment with parameters
   - Build demo interface

4. **Deploy:**
   - Use Gradio interface
   - Deploy FastAPI backend
   - Build React UI

---
# Section 4: Model Evaluation <a id="section-4"></a>

In this section, we evaluate the fine-tuned model's performance:

**Evaluation approach:**
- Compare base TinyLlama vs fine-tuned model
- Calculate quantitative metrics (BLEU, ROUGE)
- Perform qualitative analysis
- Visualize improvements

**Metrics used:**
- **BLEU:** Measures n-gram overlap between generated and reference text
- **ROUGE-1:** Unigram overlap
- **ROUGE-L:** Longest common subsequence

**Expected results:**
- Fine-tuned model should outperform base model
- Higher BLEU/ROUGE scores = better quality
- Qualitative improvements in medical accuracy

## 4.1 Import Libraries and Setup

**What this does:** Imports required libraries for evaluation.

**Libraries:**
- `torch`: For model inference
- `transformers`: Load models
- `peft`: Load LoRA adapters
- `evaluate`: HuggingFace metrics library
- `pandas`: Data handling

**Device:** Uses GPU if available, CPU otherwise (GPU much faster)

In [ ]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from evaluate import load
import matplotlib.pyplot as plt

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('EVALUATION SETUP')
print('=' * 80)
print(f'Device: {device}')
print('Libraries loaded successfully!')

## 4.2 Load Base Model

**What this does:** Loads the original TinyLlama model without fine-tuning.

**Purpose:** Establish baseline performance to compare against fine-tuned model.

**Configuration:**
- No quantization (for fair comparison)
- Float16 precision for speed
- Automatic device placement

In [ ]:
MODEL_NAME = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

print('Loading base model...')

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    torch_dtype=torch.float16
)

print('\nBASE MODEL LOADED')
print('=' * 80)
print(f'Model: {MODEL_NAME}')
print(f'Parameters: {base_model.num_parameters():,}')
print('This is the ORIGINAL model without medical fine-tuning.')

## 4.3 Load Fine-tuned Model

**What this does:** Loads the fine-tuned model with LoRA adapters.

**Process:**
1. Load base TinyLlama model
2. Load LoRA adapters from saved directory
3. Merge adapters with base model

**merge_and_unload():** Combines LoRA adapters with base weights for faster inference.

**Location:** Loads from `../models/final/medical_llm_final` or `./medical_llm_final`

In [ ]:
print('Loading fine-tuned model...')

# Load base model again
finetuned_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    torch_dtype=torch.float16
)

# Load LoRA adapters
finetuned_model = PeftModel.from_pretrained(
    finetuned_model,
    './medical_llm_final'  # Adjust path if needed
)

# Merge adapters for faster inference
finetuned_model = finetuned_model.merge_and_unload()

print('\nFINE-TUNED MODEL LOADED')
print('=' * 80)
print('LoRA adapters merged with base model.')
print('This model has been trained on medical Q&A data.')

## 4.4 Load Test Dataset

**What this does:** Loads the test set for evaluation.

**Test set:**
- 250 medical Q&A pairs (5% of total)
- Never seen during training
- Provides unbiased performance estimate

**Subset selection:** Evaluating all 250 takes time, so we evaluate 50 samples.
- Still statistically significant
- Balances thoroughness with speed
- Can evaluate full 250 if time permits

In [ ]:
# Load test data
test_df = pd.read_csv('../data/processed/test.csv')

# Select 50 samples for evaluation (adjust as needed)
test_questions = test_df['input'].tolist()[:50]
references = test_df['output'].tolist()[:50]

print('TEST DATA LOADED')
print('=' * 80)
print(f'Total test examples: {len(test_df)}')
print(f'Evaluating on: {len(test_questions)} samples')
print(f'\nSample question: {test_questions[0]}')
print(f'Reference answer: {references[0]}')

## 4.5 Generate Predictions

**What this does:** Generates responses from both models for all test questions.

**Process:**
1. For each test question:
   - Format as instruction prompt
   - Generate response from base model
   - Generate response from fine-tuned model
2. Collect all predictions

**Generation parameters:**
- `max_new_tokens=150`: Limit response length
- `temperature=0.7`: Balance between focused and creative
- `do_sample=True`: Use sampling for variety

**Duration:** Takes 5-10 minutes for 50 samples (faster with GPU)

In [ ]:
def generate_response(model, question):
    """
    Generate a response to a medical question.
    
    Args:
        model: The model to use for generation
        question: The medical question string
    
    Returns:
        Generated response string
    """
    # Format prompt
    prompt = f"""### Instruction:
{question}

### Response:
"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,
            temperature=0.7,
            do_sample=True
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the response part
    if '### Response:' in response:
        response = response.split('### Response:')[1].strip()
    
    return response

print('GENERATING PREDICTIONS')
print('=' * 80)
print('This will take 5-10 minutes...')
print()

# Generate predictions from base model
print('Generating base model predictions...')
base_predictions = [generate_response(base_model, q) for q in test_questions]

# Generate predictions from fine-tuned model
print('Generating fine-tuned model predictions...')
finetuned_predictions = [generate_response(finetuned_model, q) for q in test_questions]

print('\nPREDICTIONS COMPLETE!')
print('=' * 80)
print(f'Base model predictions: {len(base_predictions)}')
print(f'Fine-tuned predictions: {len(finetuned_predictions)}')
print('Ready for metric calculation!')

## 4.6 Calculate Evaluation Metrics

**What this does:** Computes BLEU and ROUGE scores for both models.

**Metrics explained:**

**BLEU (Bilingual Evaluation Understudy):**
- Measures n-gram overlap between generated and reference text
- Range: 0.0 (worst) to 1.0 (perfect match)
- Higher = more similar to reference
- Good for measuring factual accuracy

**ROUGE-1 (Recall-Oriented Understudy for Gisting Evaluation):**
- Measures unigram (single word) overlap
- Range: 0.0 to 1.0
- Checks if important words are present

**ROUGE-L:**
- Measures longest common subsequence
- Captures sentence-level structure
- Better for evaluating fluency

**Expected improvement:** Fine-tuned model should score 10-30% higher on all metrics.

In [ ]:
# Load metrics
bleu = load('bleu')
rouge = load('rouge')

# Debug: Check predictions before calculating metrics
print('=' * 80)
print('PREDICTION DEBUGGING')
print('=' * 80)
print(f'Number of test questions: {len(test_questions)}')
print(f'Number of references: {len(references)}')
print(f'Number of base predictions: {len(base_predictions)}')
print(f'Number of finetuned predictions: {len(finetuned_predictions)}')
print(f'\nSample check (first 3 examples):')
for i in range(min(3, len(test_questions))):
    print(f'\n--- Example {i+1} ---')
    print(f'Question: {test_questions[i][:100]}...')
    print(f'Reference length: {len(references[i])} chars')
    print(f'Base pred length: {len(base_predictions[i])} chars')
    print(f'Finetuned pred length: {len(finetuned_predictions[i])} chars')
    print(f'Base prediction: {base_predictions[i][:150]}...')
    print(f'Finetuned prediction: {finetuned_predictions[i][:150]}...')

print('\n' + '=' * 80)
print('Calculating metrics...')

# Calculate BLEU scores
base_bleu = bleu.compute(
    predictions=base_predictions, 
    references=[[r] for r in references]  # BLEU expects list of references
)
finetuned_bleu = bleu.compute(
    predictions=finetuned_predictions, 

    references=[[r] for r in references]
)

# Calculate ROUGE scores
base_rouge = rouge.compute(
    predictions=base_predictions, 
    references=references
)
finetuned_rouge = rouge.compute(
    predictions=finetuned_predictions, 
    references=references
)

# Display results
print('\n' + '=' * 80)
print('EVALUATION RESULTS')
print('=' * 80)

# Helper function to calculate percentage improvement safely
def calculate_improvement_pct(base_val, finetuned_val):
    if base_val == 0:
        if finetuned_val == 0:
            return 0.0
        else:
            return float('inf')  # Infinite improvement from 0
    return ((finetuned_val - base_val) / base_val * 100)

print('\nBLEU Score:')
print(f'  Base Model:       {base_bleu["bleu"]:.4f}')
print(f'  Fine-tuned Model: {finetuned_bleu["bleu"]:.4f}')
bleu_diff = finetuned_bleu["bleu"] - base_bleu["bleu"]
bleu_pct = calculate_improvement_pct(base_bleu["bleu"], finetuned_bleu["bleu"])
if bleu_pct == float('inf'):
    print(f'  Improvement:      +{bleu_diff:.4f} (∞% - base was 0)')
else:
    print(f'  Improvement:      +{bleu_diff:.4f} ({bleu_pct:.1f}%)')

print('\nROUGE-1 (Unigram Overlap):')
print(f'  Base Model:       {base_rouge["rouge1"]:.4f}')
print(f'  Fine-tuned Model: {finetuned_rouge["rouge1"]:.4f}')
rouge1_diff = finetuned_rouge["rouge1"] - base_rouge["rouge1"]
rouge1_pct = calculate_improvement_pct(base_rouge["rouge1"], finetuned_rouge["rouge1"])
if rouge1_pct == float('inf'):
    print(f'  Improvement:      +{rouge1_diff:.4f} (∞% - base was 0)')
else:
    print(f'  Improvement:      +{rouge1_diff:.4f} ({rouge1_pct:.1f}%)')

print('\nROUGE-L (Longest Common Subsequence):')
print(f'  Base Model:       {base_rouge["rougeL"]:.4f}')
print(f'  Fine-tuned Model: {finetuned_rouge["rougeL"]:.4f}')
rougeL_diff = finetuned_rouge["rougeL"] - base_rouge["rougeL"]
rougeL_pct = calculate_improvement_pct(base_rouge["rougeL"], finetuned_rouge["rougeL"])
if rougeL_pct == float('inf'):
    print(f'  Improvement:      +{rougeL_diff:.4f} (∞% - base was 0)')
else:
    print(f'  Improvement:      +{rougeL_diff:.4f} ({rougeL_pct:.1f}%)')

print('\n' + '=' * 80)
# Prepare metrics dictionaries for comprehensive visualization
base_metrics = {
    'bleu': base_bleu['bleu'],
    'bleu_1': base_bleu.get('precisions', [0, 0, 0, 0])[0],
    'bleu_2': base_bleu.get('precisions', [0, 0, 0, 0])[1],
    'bleu_3': base_bleu.get('precisions', [0, 0, 0, 0])[2],
    'bleu_4': base_bleu.get('precisions', [0, 0, 0, 0])[3],
    'rouge_1': base_rouge['rouge1'],
    'rouge_2': base_rouge['rouge2'],
    'rouge_l': base_rouge['rougeL']
}

finetuned_metrics = {
    'bleu': finetuned_bleu['bleu'],
    'bleu_1': finetuned_bleu.get('precisions', [0, 0, 0, 0])[0],
    'bleu_2': finetuned_bleu.get('precisions', [0, 0, 0, 0])[1],
    'bleu_3': finetuned_bleu.get('precisions', [0, 0, 0, 0])[2],
    'bleu_4': finetuned_bleu.get('precisions', [0, 0, 0, 0])[3],
    'rouge_1': finetuned_rouge['rouge1'],
    'rouge_2': finetuned_rouge['rouge2'],
    'rouge_l': finetuned_rouge['rougeL']
}

print('\n✓ Metrics dictionaries created for visualization')

In [ ]:
# Create comprehensive evaluation visualizations
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

fig.suptitle('Model Evaluation - Base vs Fine-tuned Comparison', fontsize=18, fontweight='bold')

# Prepare data
metrics_names = list(base_metrics.keys())
base_values = [base_metrics[m] for m in metrics_names]
finetuned_values = [finetuned_metrics[m] for m in metrics_names]
improvements = [((ft - base) / base * 100) if base > 0 else 0 
                for base, ft in zip(base_values, finetuned_values)]

# Color palette
colors_base = '#ff6b6b'
colors_ft = '#4ecdc4'
colors_improvement = '#95e1d3'

# 1. Overall Metrics Comparison (Bar Chart)
ax1 = fig.add_subplot(gs[0, :2])
x = np.arange(len(metrics_names))
width = 0.35

bars1 = ax1.bar(x - width/2, base_values, width, label='Base Model', 
                color=colors_base, alpha=0.8, edgecolor='black', linewidth=1.5)
bars2 = ax1.bar(x + width/2, finetuned_values, width, label='Fine-tuned Model',
                color=colors_ft, alpha=0.8, edgecolor='black', linewidth=1.5)

ax1.set_xlabel('Metrics', fontsize=12, fontweight='bold')
ax1.set_ylabel('Score', fontsize=12, fontweight='bold')
ax1.set_title('Base vs Fine-tuned Model - All Metrics', fontsize=14, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels([m.upper().replace('_', '-') for m in metrics_names], rotation=45, ha='right')
ax1.legend(fontsize=11, loc='upper left')
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

# 2. Improvement Percentage (Horizontal Bar Chart)
ax2 = fig.add_subplot(gs[0, 2])
colors_imp = ['green' if imp > 0 else 'red' for imp in improvements]
bars = ax2.barh(range(len(metrics_names)), improvements, color=colors_imp, alpha=0.7, edgecolor='black')
ax2.set_yticks(range(len(metrics_names)))
ax2.set_yticklabels([m.upper().replace('_', '-') for m in metrics_names], fontsize=10)
ax2.set_xlabel('Improvement (%)', fontsize=11, fontweight='bold')
ax2.set_title('Metric Improvements', fontsize=12, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='--', linewidth=1)
ax2.grid(axis='x', alpha=0.3, linestyle='--')

# Add percentage labels
for i, (bar, imp) in enumerate(zip(bars, improvements)):
    width = bar.get_width()
    label_x = width + (1 if width > 0 else -1)
    ax2.text(label_x, bar.get_y() + bar.get_height()/2,
            f'{imp:+.1f}%', ha='left' if width > 0 else 'right', 
            va='center', fontsize=9, fontweight='bold')

# 3. BLEU Scores Detailed Comparison
ax3 = fig.add_subplot(gs[1, 0])
bleu_metrics = [m for m in metrics_names if 'bleu' in m.lower()]
bleu_base = [base_metrics[m] for m in bleu_metrics]
bleu_ft = [finetuned_metrics[m] for m in bleu_metrics]

x_bleu = np.arange(len(bleu_metrics))
ax3.plot(x_bleu, bleu_base, marker='o', linewidth=2, markersize=8, 
         label='Base Model', color=colors_base)
ax3.plot(x_bleu, bleu_ft, marker='s', linewidth=2, markersize=8,
         label='Fine-tuned', color=colors_ft)
ax3.set_xticks(x_bleu)
ax3.set_xticklabels([m.replace('bleu_', 'BLEU-').upper() for m in bleu_metrics])
ax3.set_ylabel('Score', fontsize=11, fontweight='bold')
ax3.set_title('BLEU Scores Comparison', fontsize=12, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3, linestyle='--')

# 4. ROUGE Scores Detailed Comparison
ax4 = fig.add_subplot(gs[1, 1])
rouge_metrics = [m for m in metrics_names if 'rouge' in m.lower()]
rouge_base = [base_metrics[m] for m in rouge_metrics]
rouge_ft = [finetuned_metrics[m] for m in rouge_metrics]

x_rouge = np.arange(len(rouge_metrics))
ax4.plot(x_rouge, rouge_base, marker='o', linewidth=2, markersize=8,
         label='Base Model', color=colors_base)
ax4.plot(x_rouge, rouge_ft, marker='s', linewidth=2, markersize=8,
         label='Fine-tuned', color=colors_ft)
ax4.set_xticks(x_rouge)
ax4.set_xticklabels([m.replace('rouge_', 'ROUGE-').replace('_', '-').upper() for m in rouge_metrics],
                     rotation=45, ha='right')
ax4.set_ylabel('Score', fontsize=11, fontweight='bold')
ax4.set_title('ROUGE Scores Comparison', fontsize=12, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3, linestyle='--')

# 5. Average Score Comparison (Gauge-style)
ax5 = fig.add_subplot(gs[1, 2])
avg_base = np.mean(base_values)
avg_ft = np.mean(finetuned_values)
avg_improvement = ((avg_ft - avg_base) / avg_base * 100) if avg_base > 0 else 0

categories = ['Base\nModel', 'Fine-tuned\nModel']
averages = [avg_base, avg_ft]
colors_avg = [colors_base, colors_ft]

bars = ax5.bar(categories, averages, color=colors_avg, alpha=0.8, edgecolor='black', linewidth=2)
ax5.set_ylabel('Average Score', fontsize=11, fontweight='bold')
ax5.set_title('Overall Average Performance', fontsize=12, fontweight='bold')
ax5.set_ylim(0, max(averages) * 1.2)
ax5.grid(axis='y', alpha=0.3, linestyle='--')

for bar, avg in zip(bars, averages):
    height = bar.get_height()
    ax5.text(bar.get_x() + bar.get_width()/2., height,
            f'{avg:.4f}', ha='center', va='bottom', fontsize=11, fontweight='bold')

# Add improvement annotation
ax5.text(0.5, max(averages) * 1.1, f'Improvement: {avg_improvement:+.2f}%',
        ha='center', fontsize=11, fontweight='bold',
        bbox=dict(boxstyle='round', facecolor='yellow', alpha=0.7))

# 6. Summary Statistics Table
ax6 = fig.add_subplot(gs[2, :])
ax6.axis('tight')
ax6.axis('off')

table_data = []
table_data.append(['Metric', 'Base Model', 'Fine-tuned Model', 'Improvement', 'Status'])
for i, metric in enumerate(metrics_names):
    status = '✓ Better' if improvements[i] > 0 else '✗ Worse' if improvements[i] < 0 else '= Same'
    status_color = 'lightgreen' if improvements[i] > 0 else 'lightcoral' if improvements[i] < 0 else 'lightyellow'
    table_data.append([
        metric.upper().replace('_', '-'),
        f'{base_values[i]:.4f}',
        f'{finetuned_values[i]:.4f}',
        f'{improvements[i]:+.2f}%',
        status
    ])

# Add average row
table_data.append(['AVERAGE', f'{avg_base:.4f}', f'{avg_ft:.4f}', 
                   f'{avg_improvement:+.2f}%', '✓ Better' if avg_improvement > 0 else '✗ Worse'])

table = ax6.table(cellText=table_data, cellLoc='center', loc='center',
                  colWidths=[0.25, 0.15, 0.15, 0.15, 0.15])
table.auto_set_font_size(False)
table.set_fontsize(10)
table.scale(1, 2)

# Style the header row
for i in range(5):
    table[(0, i)].set_facecolor('#40466e')
    table[(0, i)].set_text_props(weight='bold', color='white')

# Style data rows
for i in range(1, len(table_data)):
    for j in range(5):
        if i == len(table_data) - 1:  # Average row
            table[(i, j)].set_facecolor('#fffacd')
            table[(i, j)].set_text_props(weight='bold')
        elif j == 4:  # Status column
            if 'Better' in table_data[i][4]:
                table[(i, j)].set_facecolor('lightgreen')
            elif 'Worse' in table_data[i][4]:
                table[(i, j)].set_facecolor('lightcoral')
        else:
            table[(i, j)].set_facecolor('white' if i % 2 == 0 else '#f0f0f0')

plt.tight_layout()

# Save comprehensive evaluation visualization
save_path = 'results/visualizations/evaluation/comprehensive_evaluation.png'
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f'✓ Saved: {save_path}')
plt.show()

# Create and save detailed evaluation results table
eval_results_df = pd.DataFrame({
    'Metric': [m.upper().replace('_', '-') for m in metrics_names],
    'Base_Model': base_values,
    'Finetuned_Model': finetuned_values,
    'Improvement_Percent': improvements,
    'Improvement_Absolute': [ft - base for base, ft in zip(base_values, finetuned_values)]
})

# Add summary row
summary_row = pd.DataFrame({
    'Metric': ['AVERAGE'],
    'Base_Model': [avg_base],
    'Finetuned_Model': [avg_ft],
    'Improvement_Percent': [avg_improvement],
    'Improvement_Absolute': [avg_ft - avg_base]
})
eval_results_df = pd.concat([eval_results_df, summary_row], ignore_index=True)

# Save results
results_path = 'results/metrics/evaluation_results.csv'
eval_results_df.to_csv(results_path, index=False)
print(f'✓ Saved: {results_path}')

# Display table
print('\n' + '='*90)
print('COMPREHENSIVE EVALUATION RESULTS')
print('='*90)
print(eval_results_df.to_string(index=False))
print('='*90)
print(f'\n Overall Improvement: {avg_improvement:+.2f}%')
print(f' Best Improvement: {max(improvements):.2f}% ({metrics_names[improvements.index(max(improvements))].upper()})')
print('='*90)

## 4.5 Comprehensive Evaluation Visualizations

**What this does:** Creates detailed comparison visualizations:
- Base vs Fine-tuned model metrics (bar charts)
- BLEU and ROUGE score comparisons
- Metric improvement percentages
- Side-by-side response quality comparison
- Per-metric breakdown

**Visualizations saved:**
- Model comparison charts (multiple views)
- Metrics breakdown radar chart
- Improvement analysis
- Detailed evaluation results table

**Why it's important:** Visual comparison clearly shows the fine-tuning impact and helps justify the approach.

## 4.7 Visualize Metric Comparison

**What this does:** Creates a bar chart comparing base vs fine-tuned model performance.

**Visualization benefits:**
- Easy to see improvements at a glance
- Suitable for reports and presentations
- Clearly shows fine-tuned model advantage

**Chart details:**
- Blue bars: Base model
- Orange bars: Fine-tuned model
- Y-axis: Score (0-1 scale)
- X-axis: Different metrics

In [ ]:
# Prepare data for visualization
metrics = ['BLEU', 'ROUGE-1', 'ROUGE-L']
base_scores = [
    base_bleu['bleu'], 
    base_rouge['rouge1'], 
    base_rouge['rougeL']
]
finetuned_scores = [
    finetuned_bleu['bleu'], 
    finetuned_rouge['rouge1'], 
    finetuned_rouge['rougeL']
]

# Create bar chart
x = range(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))

# Plot bars
bars1 = ax.bar(
    [i - width/2 for i in x], 
    base_scores, 
    width, 
    label='Base Model', 
    alpha=0.8,
    color='skyblue'
)
bars2 = ax.bar(
    [i + width/2 for i in x], 
    finetuned_scores, 
    width, 
    label='Fine-tuned Model', 
    alpha=0.8,
    color='lightcoral'
)

# Customize chart
ax.set_xlabel('Metrics', fontsize=12)
ax.set_ylabel('Score', fontsize=12)
ax.set_title('Model Performance Comparison', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, max(max(base_scores), max(finetuned_scores)) * 1.1)

# Add value labels on bars
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(
            bar.get_x() + bar.get_width()/2., 
            height,
            f'{height:.3f}',
            ha='center', 
            va='bottom',
            fontsize=9
        )

plt.tight_layout()
plt.savefig('../evaluation/comparisons/metrics_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print('Visualization saved to: ../evaluation/comparisons/metrics_comparison.png')

## 4.8 Qualitative Comparison - Example Responses

**What this does:** Shows side-by-side comparison of responses from both models.

**Purpose:**
- Metrics don't tell the whole story
- Human evaluation is crucial
- Shows qualitative improvements in:
  - Medical accuracy
  - Response relevance
  - Completeness
  - Clarity

**What to look for:**
- Fine-tuned model should give more medically accurate answers
- Responses should be more focused on the question
- Better use of medical terminology

In [ ]:
print('QUALITATIVE COMPARISON - SAMPLE RESPONSES')
print('=' * 80)

# Determine how many examples to display (up to 3)
num_examples = min(3, len(test_questions), len(references), len(base_predictions), len(finetuned_predictions))

if num_examples == 0:
    print('\nNo predictions available. Please run the prediction generation cells first.')
else:
    # Display example comparisons
    for i in range(num_examples):
        print(f'\nExample {i+1}:')
        print('-' * 80)
        print(f'QUESTION: {test_questions[i]}')
        print()
        print(f'REFERENCE ANSWER:')
        print(f'{references[i]}')
        print()
        print(f'BASE MODEL RESPONSE:')
        print(f'{base_predictions[i]}')
        print()
        print(f'FINE-TUNED MODEL RESPONSE:')
        print(f'{finetuned_predictions[i]}')
        print('=' * 80)

    print('\nQualitative Analysis:')
    print('- Compare medical accuracy of responses')
    print('- Fine-tuned model should be more relevant and focused')
    print('- Better use of medical terminology')

## 4.9 Save Evaluation Results

**What this does:** Saves all metrics to a JSON file for later reference.

**File:** `../evaluation/metrics/evaluation_results.json`

**Contents:**
- BLEU scores (base and fine-tuned)
- ROUGE-1 scores
- ROUGE-L scores

**Use cases:**
- Include in project report
- Compare with future training runs
- Track model improvements over time

In [ ]:
import json
import os

# Prepare results dictionary
results = {
    'base_bleu': float(base_bleu['bleu']),
    'finetuned_bleu': float(finetuned_bleu['bleu']),
    'bleu_improvement': float(finetuned_bleu['bleu'] - base_bleu['bleu']),
    'base_rouge1': float(base_rouge['rouge1']),
    'finetuned_rouge1': float(finetuned_rouge['rouge1']),
    'rouge1_improvement': float(finetuned_rouge['rouge1'] - base_rouge['rouge1']),
    'base_rougeL': float(base_rouge['rougeL']),
    'finetuned_rougeL': float(finetuned_rouge['rougeL']),
    'rougeL_improvement': float(finetuned_rouge['rougeL'] - base_rouge['rougeL']),
    'num_test_samples': len(test_questions)
}

# Create directory
os.makedirs('../evaluation/metrics', exist_ok=True)

# Save to JSON
with open('../evaluation/metrics/evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print('RESULTS SAVED')
print('=' * 80)
print('File: ../evaluation/metrics/evaluation_results.json')
print('\nContents:')
print(json.dumps(results, indent=2))
print('\nEvaluation complete! Results ready for report.')

In [ ]:
# Create example predictions comparison visualization
from textwrap import wrap

# Select diverse test examples based on available data
dataset_size = len(test_dataset)
num_examples = min(5, dataset_size)  # Up to 5 examples, or less if dataset is smaller

if dataset_size == 0:
    print('WARNING: test_dataset is empty. Cannot generate example comparisons.')
    example_comparisons = []
else:
    # Generate evenly spaced indices across the dataset
    if num_examples == 1:
        example_indices = [0]
    else:
        step = (dataset_size - 1) // (num_examples - 1)
        example_indices = [i * step for i in range(num_examples)]
        # Ensure last index is within bounds
        example_indices[-1] = min(example_indices[-1], dataset_size - 1)
    
    print(f'Dataset size: {dataset_size}')
    print(f'Selected {num_examples} examples at indices: {example_indices}')
    
    examples = [test_dataset[i] for i in example_indices]

    # Generate predictions
    print('Generating predictions for comparison...')
    example_comparisons = []

    for idx, example in enumerate(examples):
        # Extract question
        question = example['text'].split('### Response:')[0].replace('### Instruction:', '').strip()
        expected = example['text'].split('### Response:')[1].strip() if '### Response:' in example['text'] else ''
        
        # Generate from base model
        inputs_base = tokenizer(question, return_tensors='pt', truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs_base = base_model.generate(**inputs_base, max_new_tokens=100, do_sample=False)
        response_base = tokenizer.decode(outputs_base[0], skip_special_tokens=True).replace(question, '').strip()
        
        # Generate from fine-tuned model
        inputs_ft = tokenizer(question, return_tensors='pt', truncation=True, max_length=512).to(device)
        with torch.no_grad():
            outputs_ft = model.generate(**inputs_ft, max_new_tokens=100, do_sample=False)
        response_ft = tokenizer.decode(outputs_ft[0], skip_special_tokens=True).replace(question, '').strip()
        
        example_comparisons.append({
            'question': question[:100] + '...' if len(question) > 100 else question,
            'expected': expected[:100] + '...' if len(expected) > 100 else expected,
            'base': response_base[:100] + '...' if len(response_base) > 100 else response_base,
            'finetuned': response_ft[:100] + '...' if len(response_ft) > 100 else response_ft
        })

    print(f' Generated {len(example_comparisons)} example comparisons')

# Only create visualizations if we have examples
if len(example_comparisons) > 0:
    # Create comparison table visualization
    fig, axes = plt.subplots(len(example_comparisons), 1, figsize=(18, 5 * len(example_comparisons)))
    if len(example_comparisons) == 1:
        axes = [axes]

    fig.suptitle('Example Predictions - Base vs Fine-tuned Model Comparison', 
                 fontsize=16, fontweight='bold', y=0.995)

    for idx, (ax, comparison) in enumerate(zip(axes, example_comparisons)):
        ax.axis('tight')
        ax.axis('off')
        
        # Create table data
        table_data = [
            ['Component', 'Content'],
            ['Question', '\n'.join(wrap(comparison['question'], width=80))],
            ['Expected Answer', '\n'.join(wrap(comparison['expected'], width=80))],
            ['Base Model', '\n'.join(wrap(comparison['base'], width=80))],
            ['Fine-tuned Model', '\n'.join(wrap(comparison['finetuned'], width=80))]
        ]
        
        table = ax.table(cellText=table_data, cellLoc='left', loc='center',
                         colWidths=[0.2, 0.8])
        table.auto_set_font_size(False)
        table.set_fontsize(9)
        table.scale(1, 3)
        
        # Style the table
        table[(0, 0)].set_facecolor('#40466e')
        table[(0, 1)].set_facecolor('#40466e')
        table[(0, 0)].set_text_props(weight='bold', color='white')
        table[(0, 1)].set_text_props(weight='bold', color='white')
        
        colors = ['#e8f4f8', '#fff9e6', '#ffe6e6', '#e6ffe6']
        for i in range(1, 5):
            table[(i, 0)].set_facecolor('#f0f0f0')
            table[(i, 0)].set_text_props(weight='bold')
            table[(i, 1)].set_facecolor(colors[i-1])
        
        ax.set_title(f'Example {idx + 1}', fontsize=12, fontweight='bold', pad=10)

    plt.tight_layout()

    # Save examples visualization
    save_path = 'results/visualizations/evaluation/example_predictions.png'
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f'✓ Saved: {save_path}')
    plt.show()

    # Save examples as CSV
    examples_df = pd.DataFrame(example_comparisons)
    examples_df.index = [f'Example_{i+1}' for i in range(len(examples_df))]
    examples_path = 'results/metrics/example_predictions.csv'
    examples_df.to_csv(examples_path)
    print(f'✓ Saved: {examples_path}')
else:
    print('No examples to visualize.')

## 4.6 Example Predictions Visualization

**What this does:** Creates side-by-side comparison of model responses:
- Shows actual medical questions
- Displays expected answers
- Shows base model responses
- Shows fine-tuned model responses
- Highlights quality improvements

**Visualizations saved:**
- Example predictions comparison table
- Response quality analysis

**Why it's important:** Real examples demonstrate the practical improvement in response quality.

---
# Section 5: Interactive Inference Demo <a id="section-5"></a>

In this section, we demonstrate the fine-tuned model's capabilities:

**Features:**
- Load fine-tuned model
- Test with various medical questions
- Experiment with generation parameters
- Explore different medical domains
- Interactive mode for custom questions

**Goal:** Showcase the model's practical medical Q&A abilities.

## 5.1 Setup and Load Model

**What this does:** Loads the fine-tuned model for interactive testing.

**Setup:**
- Import required libraries
- Detect available device (GPU/CPU)
- Load tokenizer and model
- Merge LoRA adapters

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Detect device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('INFERENCE DEMO SETUP')
print('=' * 80)
print(f'Using device: {device}')

# Load model
MODEL_NAME = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map='auto',
    torch_dtype=torch.float16
)

model = PeftModel.from_pretrained(
    base_model,
    './medical_llm_final'
)
model = model.merge_and_unload()

print('\nModel loaded successfully!')
print('Ready for medical Q&A!')

## 5.2 Define Generation Function

**What this does:** Creates a reusable function for asking medical questions.

**Function parameters:**
- `question`: The medical question string
- `temperature`: Controls randomness (0.1-1.0)
  - Lower (0.3) = More focused, deterministic
  - Higher (1.0) = More creative, diverse
- `max_length`: Maximum response length in tokens

**Generation settings:**
- `top_p=0.9`: Nucleus sampling (consider top 90% probability mass)
- `do_sample=True`: Use sampling instead of greedy decoding
- `repetition_penalty=1.1`: Penalize repeated phrases

In [ ]:
def ask_medical_question(question, temperature=0.7, max_length=200):
    """
    Generate a response to a medical question.
    
    Args:
        question (str): The medical question
        temperature (float): Sampling temperature (0.1-1.0)
            - Lower = more focused and deterministic
            - Higher = more creative and diverse
        max_length (int): Maximum response length in tokens
    
    Returns:
        str: Generated medical response
    """
    # Format prompt
    prompt = f"""### Instruction:
{question}

### Response:
"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors='pt', truncation=True).to(device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            top_p=0.9,                # Nucleus sampling
            do_sample=True,            # Enable sampling
            repetition_penalty=1.1     # Reduce repetition
        )
    
    # Decode response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract only the response portion
    if '### Response:' in response:
        response = response.split('### Response:')[1].strip()
    
    return response

print('Generation function ready!')
print('\nUsage: ask_medical_question("Your question?", temperature=0.7, max_length=200)')

## 5.3 Test with Common Medical Questions

**What this does:** Tests the model with 5 common medical questions.

**Test questions cover:**
1. Diabetes symptoms
2. Hypertension treatment
3. Aspirin side effects
4. Migraine causes
5. Insulin mechanism

**Purpose:** Demonstrate model's ability to answer diverse medical questions.

In [ ]:
test_questions = [
    'What are the symptoms of diabetes?',
    'How is hypertension treated?',
    'What are the side effects of aspirin?',
    'What causes migraine headaches?',
    'How does insulin work?'
]

print('MEDICAL Q&A DEMONSTRATIONS')
print('=' * 80)

for i, question in enumerate(test_questions, 1):
    print(f'\nQuestion {i}: {question}')
    print('-' * 80)
    
    response = ask_medical_question(question)
    
    print(f'Answer: {response}')
    print('=' * 80)

## 5.4 Parameter Experimentation - Temperature

**What this does:** Shows how temperature affects response quality.

**Temperature values tested:**
- **0.3:** More focused, factual, deterministic responses
- **0.7:** Balanced between creativity and accuracy
- **1.0:** More creative, diverse, but potentially less accurate

**Observation:** Lower temperature is typically better for medical questions where accuracy is crucial.

In [ ]:
test_q = 'What are the symptoms of heart disease?'

print('PARAMETER EXPERIMENTATION - Temperature')
print('=' * 80)
print(f'Question: {test_q}')
print('\nTesting different temperature values...')
print()

for temp in [0.3, 0.7, 1.0]:
    print(f'\nTemperature: {temp}')
    print('-' * 80)
    response = ask_medical_question(test_q, temperature=temp)
    print(f'Response: {response}')
    print('=' * 80)

print('\nObservation:')
print('- Lower temperature (0.3): More focused, factual')
print('- Medium temperature (0.7): Balanced')
print('- Higher temperature (1.0): More creative, diverse')

In [ ]:
# Create comprehensive experiment tracking report
from datetime import datetime
import json

print('='*80)
print('CREATING COMPREHENSIVE EXPERIMENT TRACKING REPORT')
print('='*80)

# Get GPU info safely
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = f'{torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}'
else:
    gpu_name = 'None (CPU only)'
    gpu_memory = 'N/A'

# 1. EXPERIMENT CONFIGURATION TABLE
config_data = {
    'Experiment_Info': {
        'Experiment_ID': experiment_id,
        'Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'Platform': 'Google Colab',
        'GPU': gpu_name,
        'GPU_Memory_GB': gpu_memory
    },
    'Dataset_Config': {
        'Dataset_Name': 'medalpaca/medical_meadow_medical_flashcards',
        'Total_Samples': 5000,
        'Train_Samples': len(train_dataset),
        'Val_Samples': len(val_dataset),
        'Test_Samples': len(test_dataset),
        'Train_Split': '85%',
        'Val_Split': '10%',
        'Test_Split': '5%'
    },
    'Model_Config': {
        'Base_Model': 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
        'Model_Size': '1.1B parameters',
        'Fine_tuning_Method': 'LoRA (Low-Rank Adaptation)',
        'Quantization': '8-bit',
        'LoRA_Rank_r': peft_config.r,
        'LoRA_Alpha': peft_config.lora_alpha,
        'LoRA_Dropout': peft_config.lora_dropout,
        'Target_Modules': ', '.join(peft_config.target_modules)
    },
    'Training_Config': {
        'Num_Epochs': training_args.num_train_epochs,
        'Batch_Size': training_args.per_device_train_batch_size,
        'Gradient_Accumulation': training_args.gradient_accumulation_steps,
        'Effective_Batch_Size': training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps,
        'Learning_Rate': training_args.learning_rate,
        'Warmup_Steps': training_args.warmup_steps,
        'LR_Scheduler': training_args.lr_scheduler_type,
        'Max_Seq_Length': 512,
        'Optimizer': 'AdamW (paged_adamw_8bit)',
        'FP16': training_args.fp16
    },
    'Training_Results': {
        'Initial_Loss': f'{initial_loss:.6f}' if 'initial_loss' in locals() else 'N/A',
        'Final_Loss': f'{final_loss:.6f}' if 'final_loss' in locals() else 'N/A',
        'Loss_Improvement': f'{loss_improvement:.2f}%' if 'loss_improvement' in locals() else 'N/A',
        'Total_Training_Steps': train_steps[-1] if 'train_steps' in locals() and train_steps else 'N/A',
        'Training_Time': 'See Colab execution time'
    },
    'Evaluation_Results': {
        'Avg_Base_Score': f'{avg_base:.4f}' if 'avg_base' in locals() else 'N/A',
        'Avg_Finetuned_Score': f'{avg_ft:.4f}' if 'avg_ft' in locals() else 'N/A',
        'Overall_Improvement': f'{avg_improvement:.2f}%' if 'avg_improvement' in locals() else 'N/A',
        'Best_Metric': metrics_names[improvements.index(max(improvements))].upper() if 'improvements' in locals() else 'N/A',
        'Best_Improvement': f'{max(improvements):.2f}%' if 'improvements' in locals() else 'N/A'
    }
}

# Convert to DataFrame for display
config_rows = []
for category, items in config_data.items():
    config_rows.append(['', '', ''])  # Empty row as separator
    config_rows.append([f'=== {category.replace("_", " ").upper()} ===', '', ''])
    for key, value in items.items():
        config_rows.append([category, key.replace('_', ' '), str(value)])

config_df = pd.DataFrame(config_rows, columns=['Category', 'Parameter', 'Value'])

# Save configuration table
config_path = 'results/experiments/experiment_configuration.csv'
config_df.to_csv(config_path, index=False)
print(f'✓ Saved: {config_path}')

# 2. CREATE DETAILED EXPERIMENT LOG
log_content = f"""
{'='*80}
MEDICAL LLM FINE-TUNING - COMPLETE EXPERIMENT LOG
{'='*80}

EXPERIMENT IDENTIFICATION
{'='*80}
Experiment ID:      {experiment_id}
Date & Time:        {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Platform:           Google Colab
GPU:                {gpu_name}
GPU Memory:         {gpu_memory} GB

{'='*80}
DATASET CONFIGURATION
{'='*80}
Dataset:            medalpaca/medical_meadow_medical_flashcards
Total Samples:      5,000 (subset from 33,955)
Training:           {len(train_dataset):,} samples (85%)
Validation:         {len(val_dataset):,} samples (10%)
Test:               {len(test_dataset):,} samples (5%)

Data Format:        Instruction-Response pairs
Domain:             Medical Q&A
Preprocessing:      Prompt formatting, train/val/test split

{'='*80}
MODEL CONFIGURATION
{'='*80}
Base Model:         TinyLlama/TinyLlama-1.1B-Chat-v1.0
Model Size:         1.1 billion parameters
Architecture:       Causal Language Model (Chat-optimized)

Fine-tuning Method: LoRA (Low-Rank Adaptation)
- LoRA Rank (r):    {peft_config.r}
- LoRA Alpha:       {peft_config.lora_alpha}
- LoRA Dropout:     {peft_config.lora_dropout}
- Target Modules:   {', '.join(peft_config.target_modules)}
- Trainable Params: ~0.5% of total

Quantization:       8-bit (Memory optimization)
- Load in 8-bit:    True
- Double Quant:     True
- Memory Savings:   ~50% reduction

{'='*80}
TRAINING CONFIGURATION
{'='*80}
Number of Epochs:           {training_args.num_train_epochs}
Batch Size:                 {training_args.per_device_train_batch_size}
Gradient Accumulation:      {training_args.gradient_accumulation_steps}
Effective Batch Size:       {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}

Learning Rate:              {training_args.learning_rate}
LR Scheduler:               {training_args.lr_scheduler_type}
Warmup Steps:               {training_args.warmup_steps}

Optimizer:                  paged_adamw_8bit
Max Sequence Length:        512 tokens
FP16 Training:              {training_args.fp16}

Logging Steps:              {training_args.logging_steps}
Save Steps:                 {training_args.save_steps}

{'='*80}
TRAINING RESULTS
{'='*80}
Initial Loss:               {f'{initial_loss:.6f}' if 'initial_loss' in locals() else 'N/A'}
Final Loss:                 {f'{final_loss:.6f}' if 'final_loss' in locals() else 'N/A'}
Minimum Loss:               {f'{min_loss:.6f}' if 'min_loss' in locals() else 'N/A'}
Average Loss:               {f'{avg_loss:.6f}' if 'avg_loss' in locals() else 'N/A'}

Loss Improvement:           {f'{loss_improvement:.2f}%' if 'loss_improvement' in locals() else 'N/A'} {'(Reduction)' if 'loss_improvement' in locals() else ''}
Total Steps:                {f'{train_steps[-1]:,}' if 'train_steps' in locals() and train_steps else 'N/A'}

Training completed successfully ✓

{'='*80}
EVALUATION RESULTS (Base vs Fine-tuned)
{'='*80}
"""

# Add metric-by-metric comparison
if 'base_metrics' in locals() and 'finetuned_metrics' in locals():
    log_content += "\nMetric Comparison:\n"
    log_content += f"{'Metric':<20} {'Base':>12} {'Fine-tuned':>12} {'Improvement':>12}\n"
    log_content += f"{'-'*60}\n"
    for metric in metrics_names:
        base_val = base_metrics[metric]
        ft_val = finetuned_metrics[metric]
        imp = ((ft_val - base_val) / base_val * 100) if base_val > 0 else 0
        log_content += f"{metric.upper():<20} {base_val:>12.4f} {ft_val:>12.4f} {imp:>11.2f}%\n"
    
    log_content += f"{'-'*60}\n"
    log_content += f"{'AVERAGE':<20} {avg_base:>12.4f} {avg_ft:>12.4f} {avg_improvement:>11.2f}%\n"
    log_content += f"\n✓ Overall Performance Improvement: {avg_improvement:+.2f}%\n"
    log_content += f"✓ Best Performing Metric: {metrics_names[improvements.index(max(improvements))].upper()} ({max(improvements):.2f}%)\n"

log_content += f"""
{'='*80}
GENERATED FILES & OUTPUTS
{'='*80}

Visualizations:
   results/visualizations/data_exploration/comprehensive_data_analysis.png
   results/visualizations/preprocessing/data_split_analysis.png
   results/visualizations/training/training_progress.png
   results/visualizations/evaluation/comprehensive_evaluation.png
   results/visualizations/evaluation/example_predictions.png

Metrics & Tables:
   results/metrics/data_exploration_statistics.csv
   results/metrics/preprocessing_summary.csv
   results/metrics/training_metrics.csv
   results/metrics/evaluation_results.csv
   results/metrics/example_predictions.csv

Models:
   results/models/finetuned_medical_llm/ (Fine-tuned model with LoRA)
   results/models/base_model_metrics.json
   results/models/finetuned_model_metrics.json

Experiment Tracking:
   results/experiments/experiment_{experiment_id}.txt (This log)
   results/experiments/experiment_configuration.csv

{'='*80}
CONCLUSIONS & RECOMMENDATIONS
{'='*80}

1. Training Success:
   {' Model successfully fine-tuned with significant loss reduction' if 'loss_improvement' in locals() and loss_improvement > 0 else '- Check training metrics'}

2. Evaluation Results:
   {' Fine-tuned model outperforms base model across metrics' if 'avg_improvement' in locals() and avg_improvement > 0 else '- Check evaluation results'}

3. Next Steps:
   - Deploy model for inference testing
   - Integrate with FastAPI backend
   - Create React UI for user interaction
   - Prepare demo video for assignment submission
   - Document results in final report

4. Model Ready for:
   - Real-time medical Q&A
   - Integration into medical assistant application
   - Further fine-tuning with domain-specific data
   - Deployment to production environment

{'='*80}
EXPERIMENT COMPLETED SUCCESSFULLY ✓
{'='*80}
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""

# Save detailed log
log_path = f'results/experiments/experiment_{experiment_id}.txt'
with open(log_path, 'w') as f:
    f.write(log_content)
print(f'✓ Saved: {log_path}')

# Display summary
print(log_content)

# 3. CREATE EXPERIMENT SUMMARY VISUALIZATION
fig, axes = plt.subplots(2, 2, figsize=(18, 12))
fig.suptitle(f'Experiment Summary - {experiment_id}', fontsize=18, fontweight='bold')

# 1. Key Metrics Summary
ax1 = axes[0, 0]
ax1.axis('off')
summary_text = f"""
EXPERIMENT SUMMARY
{'='*50}

ID: {experiment_id}
Date: {datetime.now().strftime('%Y-%m-%d')}

MODEL: TinyLlama-1.1B + LoRA
METHOD: 8-bit Fine-tuning

DATASET: Medical Flashcards
- Training: {len(train_dataset):,} samples
- Validation: {len(val_dataset):,} samples  
- Test: {len(test_dataset):,} samples

TRAINING:
- Epochs: {training_args.num_train_epochs}
- Loss Improvement: {loss_improvement:.2f}% {'↓' if 'loss_improvement' in locals() else ''}

EVALUATION:
- Avg Improvement: {avg_improvement:.2f}% {'↑' if 'avg_improvement' in locals() and avg_improvement > 0 else ''}
- Best Metric: {metrics_names[improvements.index(max(improvements))].upper() if 'improvements' in locals() else 'N/A'}

STATUS: ✓ COMPLETED
"""
ax1.text(0.1, 0.5, summary_text, fontsize=12, family='monospace',
         verticalalignment='center',
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.3))

# 2. Files Generated Count
ax2 = axes[0, 1]
file_categories = ['Visualizations', 'Metrics', 'Models', 'Experiments']
file_counts = [5, 5, 3, 2]  # Counts of files in each category
colors_files = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12']

bars = ax2.bar(file_categories, file_counts, color=colors_files, alpha=0.7, edgecolor='black', linewidth=2)
ax2.set_ylabel('Number of Files', fontsize=12, fontweight='bold')
ax2.set_title('Generated Output Files', fontsize=14, fontweight='bold')
ax2.set_ylim(0, max(file_counts) + 2)

for bar, count in zip(bars, file_counts):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{count}', ha='center', va='bottom', fontsize=14, fontweight='bold')

# 3. Training Configuration
ax3 = axes[1, 0]
ax3.axis('tight')
ax3.axis('off')

config_table_data = [
    ['Configuration', 'Value'],
    ['LoRA Rank', str(peft_config.r)],
    ['LoRA Alpha', str(peft_config.lora_alpha)],
    ['Learning Rate', f'{training_args.learning_rate}'],
    ['Batch Size', f'{training_args.per_device_train_batch_size}'],
    ['Epochs', str(training_args.num_train_epochs)],
    ['Quantization', '8-bit'],
]

table = ax3.table(cellText=config_table_data, cellLoc='left', loc='center',
                  colWidths=[0.5, 0.5])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

table[(0, 0)].set_facecolor('#40466e')
table[(0, 1)].set_facecolor('#40466e')
table[(0, 0)].set_text_props(weight='bold', color='white')
table[(0, 1)].set_text_props(weight='bold', color='white')

for i in range(1, len(config_table_data)):
    table[(i, 0)].set_facecolor('#f0f0f0')
    table[(i, 0)].set_text_props(weight='bold')
    table[(i, 1)].set_facecolor('white')

ax3.set_title('Hyperparameters', fontsize=14, fontweight='bold', pad=20)

# 4. Quick Results
ax4 = axes[1, 1]
ax4.axis('tight')
ax4.axis('off')

results_table_data = [
    ['Metric', 'Result'],
    ['Training Loss ↓', f'{loss_improvement:.1f}%' if 'loss_improvement' in locals() else 'N/A'],
    ['Eval Improvement ↑', f'{avg_improvement:.1f}%' if 'avg_improvement' in locals() else 'N/A'],
    ['Best Metric', f'{metrics_names[improvements.index(max(improvements))].upper()}' if 'improvements' in locals() else 'N/A'],
    ['Status', '✓ SUCCESS'],
]

table2 = ax4.table(cellText=results_table_data, cellLoc='center', loc='center',
                   colWidths=[0.5, 0.5])
table2.auto_set_font_size(False)
table2.set_fontsize(11)
table2.scale(1, 2.5)

table2[(0, 0)].set_facecolor('#40466e')
table2[(0, 1)].set_facecolor('#40466e')
table2[(0, 0)].set_text_props(weight='bold', color='white')
table2[(0, 1)].set_text_props(weight='bold', color='white')

for i in range(1, len(results_table_data)):
    table2[(i, 0)].set_facecolor('#f0f0f0')
    table2[(i, 0)].set_text_props(weight='bold')
    if i == len(results_table_data) - 1:
        table2[(i, 1)].set_facecolor('lightgreen')
        table2[(i, 1)].set_text_props(weight='bold')
    else:
        table2[(i, 1)].set_facecolor('white')

ax4.set_title('Quick Results', fontsize=14, fontweight='bold', pad=20)

plt.tight_layout()

# Save experiment summary visualization
summary_viz_path = f'results/experiments/experiment_{experiment_id}_summary.png'
plt.savefig(summary_viz_path, dpi=300, bbox_inches='tight')
print(f' Saved: {summary_viz_path}')
plt.show()

print('\n' + '='*80)
print(' COMPREHENSIVE EXPERIMENT TRACKING COMPLETE')
print('='*80)
print(f'\n All results saved to: results/')
print(f' Total files generated: 15+')
print(f' Experiment ID: {experiment_id}')
print('='*80)

## Experiment Tracking - Complete Configuration & Results

**What this does:** Creates a comprehensive experiment tracking report including:
- All hyperparameters and configurations
- Complete training metrics
- Evaluation results
- Model comparison summary
- Resource usage
- File locations

**Files saved:**
- Complete experiment configuration table
- Full experiment log with all settings
- Summary report for documentation

**Why it's important:** Complete tracking enables reproducibility and comparison with future experiments.